# PREPROCESSING


## MISSING VALUE TREATMENT

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv("../input/ml-lab-exam/traindata_SJC.csv")
data_test=pd.read_csv("../input/ml-lab-exam/testdata_SJC.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data_test.info()

In [ ]:
data['MaritalStatus'].fillna(data['MaritalStatus'].mode()[0],inplace = True)
data["WeeklyWages"].fillna(data["WeeklyWages"].mean(),inplace = True)
data["HoursWorkedPerWeek"].fillna(data["WeeklyWages"].mean(),inplace = True)
data_test['MaritalStatus'].fillna(data_test['MaritalStatus'].mode()[0],inplace = True)

In [ ]:
data=data.drop(['DateReported','DateTimeOfAccident'],axis=1)
data_test=data_test.drop(['DateReported','DateTimeOfAccident'],axis=1)

In [ ]:
data.info()

### CHECKING FOR MISSING VALUES

In [ ]:
data.isnull()

In [ ]:
data.info()

In [ ]:
data.isnull().sum().count()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

## DATA VISUALISATION

In [ ]:
numdf=data.select_dtypes(exclude=object)
numdf.columns

In [ ]:
sns.boxplot(data=numdf.drop(["InitialIncurredCalimsCost","UltimateIncurredClaimCost"],axis=1))
plt.xticks(rotation=90)
plt.rcParams["figure.figsize"] = [10,10]

In [ ]:
data_test['WeeklyWages']=pd.cut(data['WeeklyWages'],bins=3, labels=['Low','Medium','High']).astype('object')
data_test['Age']=pd.cut(data['Age'],bins=3, labels=['Low','Medium','High']).astype('object')
data['WeeklyWages']=pd.cut(data['WeeklyWages'],bins=3, labels=['Low','Medium','High']).astype('object')
data['Age']=pd.cut(data['Age'],bins=3, labels=['Low','Medium','High']).astype('object')

In [ ]:
sns.heatmap(numdf.corr())

In [ ]:
sns.pairplot(data=numdf,diag_kind='kde')

In [ ]:
data.info()

In [ ]:
data["Gender"].value_counts().plot.bar()
plt.rcParams["figure.figsize"] = [5,5]

In [ ]:
data["MaritalStatus"].value_counts().plot.bar()
plt.rcParams["figure.figsize"] = [5,5]

In [ ]:
sns.catplot(data=data,col="MaritalStatus",y='UltimateIncurredClaimCost',kind="violin")

In [ ]:
sns.catplot(data=data,col="Gender",y='UltimateIncurredClaimCost',kind="violin")

## DATA ENCODING 

In [ ]:
import sklearn.preprocessing as pre

In [ ]:
le=pre.LabelEncoder()
sc=pre.StandardScaler()

In [ ]:
df_label=data.copy()
df_label.shape
df_label_test=data_test.copy()

In [ ]:
cat_df=data.select_dtypes(include="object")
cat_df1=data_test.select_dtypes(include="object")

In [ ]:
cat_df

In [ ]:
for x in cat_df.columns:
    df_label[x]=le.fit_transform(df_label[x])

In [ ]:
for x in cat_df.columns:
    df_label_test[x]=le.fit_transform(df_label_test[x])

In [ ]:
df_label.shape

In [ ]:
df_label

## FEATURE SELECTION

In [ ]:
import sklearn.preprocessing as pre
import sklearn.model_selection as ms

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

In [ ]:
# df_label1=df_label.drop(["ClaimNumber","ClaimDescription","UltimateIncurredClaimCost","HoursWorkedPerWeek"],axis=1)
# X = pre.minmax_scale(df_label1)
# y= df_label['UltimateIncurredClaimCost']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123456)

In [ ]:
df_label_test1=df_label_test.drop(["ClaimNumber","ClaimDescription","HoursWorkedPerWeek"],axis=1)
X_test = pre.minmax_scale(df_label_test1)


In [ ]:
df_label1=df_label.drop(["ClaimNumber","ClaimDescription","UltimateIncurredClaimCost","HoursWorkedPerWeek"],axis=1)
X_train = pre.minmax_scale(df_label1)
y_train= df_label['UltimateIncurredClaimCost']

In [ ]:
dtr = DecisionTreeRegressor( random_state=123456)
dtr.fit(X_train, y_train)

In [ ]:
importance=dtr.feature_importances_
importance

In [ ]:
for i,v in enumerate(importance):
    print("Feature: %0d, Score : %0.5f" %(i,v))

In [ ]:
sorted_idx = dtr.feature_importances_.argsort()
plt.bar(df_label1.columns[sorted_idx], dtr.feature_importances_[sorted_idx])
plt.xlabel("CART Classification Feature Importance")
plt.xticks(rotation=90)
plt.ylim(0,0.01)
plt.tick_params(axis='x', which='major', labelsize=12,length=10)
plt.rcParams["figure.figsize"] = [20,20]

In [ ]:
list_feature=df_label1.columns[sorted_idx].tolist()
list_feature

## PARAMETER TUNING

In [ ]:
from sklearn import tree, metrics, model_selection, preprocessing

In [ ]:
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import sklearn.linear_model as lm
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error

In [ ]:
xgb= xg.XGBRegressor(max_depth=8,min_child_weight=20,eta=0.8,subsample=1,colsample_bytree=1,objective='reg:linear',n_jobs=4,random_state=123,n_estimators=5)

In [ ]:
lasso=lm.Lasso(alpha=0.9, fit_intercept=True, normalize=False, precompute=False, copy_X=True, max_iter=10000, tol=0.0001, warm_start=False, positive=False, random_state=12345, selection='random')


In [ ]:
ridge=lm.Ridge(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto', random_state=None)

In [ ]:
rfr=RandomForestRegressor(n_estimators=10,random_state=1234)

In [ ]:
#lasso
lasso.fit(X_train,y_train)
#  test_score=lasso.score(X_test,y_test)
train_score=lasso.score(X_train,y_train)
y_pred_lasso=lasso.predict(X_test) 
# print("\nThe train score is:" ,train_score)

In [ ]:
#xgboost
xgb.fit(X_train,y_train)
#test_score=xgb.score(X_test,y_test)
train_score=xgb.score(X_train,y_train)
y_pred_xgb=xgb.predict(X_test)
#print("\nThe test score is:" ,test_score)
print("\nThe train score is:" ,train_score)

In [ ]:
#random forest
rfr.fit(X_train,y_train)
#test_score=xgb.score(X_test,y_test)
train_score=rfr.score(X_train,y_train)
y_pred_rfr=rfr.predict(X_test)
#print("\nThe test score is:" ,test_score)
print("\nThe train score is:" ,train_score)

In [ ]:
#ridge
ridge.fit(X_train,y_train)
#test_score=xgb.score(X_test,y_test)
train_score=ridge.score(X_train,y_train)
y_pred_ridge=ridge.predict(X_test)
#print("\nThe test score is:" ,test_score)
print("\nThe train score is:" ,train_score)

In [ ]:
#grid search for linear regression hyper parameter tuning

In [ ]:
lr1=lm.LinearRegression()

In [ ]:
 params = {"fit_intercept":[True,False],
    "normalize":[True,False],
    "copy_X":[True,False],
    "n_jobs":[2,3,4,5,6,7,8]}
params

In [ ]:
grid=ms.GridSearchCV(lr1,params)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
pd.DataFrame(grid.cv_results_)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
lr=lm.LinearRegression(copy_X=True,fit_intercept=True,n_jobs=2,normalize=False)

In [ ]:
lr.fit(X_train,y_train)
# test_score=lr.score(X_test,y_test)
train_score=lr.score(X_train,y_train)
y_pred_lr=lr.predict(X_test)
#print("\nThe test score is:" ,test_score)
print("\nThe train score is:" ,train_score)

In [ ]:
df_csv=pd.read_csv("../input/ml-lab-exam/sample_submission_csv.csv")
df_csv["UltimateIncurredClaimCost"]=y_pred_xgb
df_csv.to_csv("Sample Submission_lr2.csv", index = False)

## 